# Merge all auctions into one

Combines several datasets and do some simple analyses.  


### User variables

In [1]:
import drz_config
cfg = drz_config.read_config()
DATE = cfg['DATE']
VERBOSE = cfg['VERBOSE']
OPBOD = cfg['OPBOD']
SKIPSAVE = cfg['SKIPSAVE']

if VERBOSE > 0:
    display(cfg)

{'settings_fn': '../code/assets/drz-auction-settings.ini',
 'DATE': '2021-05',
 'VERBOSE': 1,
 'OPBOD': False,
 'URL': 'http://verkoop.domeinenrz.nl/verkoop_bij_inschrijving_2021-0005',
 'EXTEND_URL': False,
 'CLOSEDDATA': True,
 'closed_data_fields': '*',
 'SKIPSAVE': False}

In [2]:
# dates of auction
# format: yyyy-mm; yyyy: year, mm: month (%Y-%m)
if OPBOD:
    AuctionDates = ['2019-11', '2019-12', 
                    '2020-01', '2020-02', '2020-03', 
                   ]
else:    
    AuctionDates = ['2017-03', '2017-04', '2017-05', '2017-06', '2017-07', '2017-08', '2017-09', '2017-10', '2017-11', '2017-12', 
                    '2018-01', '2018-02', '2018-03', '2018-04', '2018-05', '2018-06', '2018-07', '2018-08', '2018-09', '2018-10', '2018-11', '2018-12',
                    '2019-01', '2019-02', '2019-03', '2019-04', '2019-05', '2019-06', '2019-07', '2019-08', '2019-09', '2019-10', '2019-11', '2019-12',
                    '2020-01', '2020-02', '2020-03', '2020-06', '2020-07', '2020-08', '2020-09', '2020-10', '2020-11', '2020-12',
                    '2021-01', '2021-02', '2021-03', '2021-04', '2021-05', 
                   ]  
if DATE not in AuctionDates:
    raise ValueError(f'{DATE} not in list that will be concatenated. Add to list.')

### Import modules 

In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import re
import os
plt.style.use('ggplot')

### Load all data

Auctions are saved monthly in a .pkl file

In [4]:
print('load data')
data = dict()
for AuctionDate in AuctionDates:
    # read data
    if OPBOD:
        fn = '../../../python-nb/data/rdw-data-opbod-{:s}.pkl'.format(
            AuctionDate
        )

    else:    
        fn = '../data/rdw-data-{:s}.pkl'.format(
            AuctionDate
        )

    if VERBOSE > 0: print(fn)
    df = pd.read_pickle(fn)
    data[AuctionDate] = df

if VERBOSE > 0:
    print('\nlast rows of last file')
    display(data[AuctionDates[-1]].tail())

load data
../data/rdw-data-2017-03.pkl
../data/rdw-data-2017-04.pkl
../data/rdw-data-2017-05.pkl
../data/rdw-data-2017-06.pkl
../data/rdw-data-2017-07.pkl
../data/rdw-data-2017-08.pkl
../data/rdw-data-2017-09.pkl
../data/rdw-data-2017-10.pkl
../data/rdw-data-2017-11.pkl
../data/rdw-data-2017-12.pkl
../data/rdw-data-2018-01.pkl
../data/rdw-data-2018-02.pkl
../data/rdw-data-2018-03.pkl
../data/rdw-data-2018-04.pkl
../data/rdw-data-2018-05.pkl
../data/rdw-data-2018-06.pkl
../data/rdw-data-2018-07.pkl
../data/rdw-data-2018-08.pkl
../data/rdw-data-2018-09.pkl
../data/rdw-data-2018-10.pkl
../data/rdw-data-2018-11.pkl
../data/rdw-data-2018-12.pkl
../data/rdw-data-2019-01.pkl
../data/rdw-data-2019-02.pkl
../data/rdw-data-2019-03.pkl
../data/rdw-data-2019-04.pkl
../data/rdw-data-2019-05.pkl
../data/rdw-data-2019-06.pkl
../data/rdw-data-2019-07.pkl
../data/rdw-data-2019-08.pkl
../data/rdw-data-2019-09.pkl
../data/rdw-data-2019-10.pkl
../data/rdw-data-2019-11.pkl
../data/rdw-data-2019-12.pkl
../d

,Source,Title,Price,Draw,Raw_text,N_images,Images,lot_counter,Note,LotNr,...,rdw_ovi_aantal_staanplaatsen,rdw_ovi_afwijkende_max_snelheid,rdw_ovi_laadvermogen,rdw_ovi_lengte,rdw_ovi_sub_cat_omschrijving,rdw_ovi_afstand_hartkoppeling_tot_achterzijde_vrtg,rdw_ovi_private_owners,rdw_ovi_company_owner,rdw_ovi_owners,rdw_ovi_under_survey
2021-05-8158,http://verkoop.domeinenrz.nl/verkoop_bij_insch...,Kavel K2100058158,2222.0,False,"[Type range rover sport, Roemeens kenteken, Au...",13,[http://verkoop.domeinenrz.nl/ufc/static/16194...,8158,False,K2100058158,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2021-05-8159,http://verkoop.domeinenrz.nl/verkoop_bij_insch...,Kavel K2100058159,4919.0,False,"[Type XD100P2, Serienummer U519856U, Afgelezen...",9,[http://verkoop.domeinenrz.nl/ufc/static/16194...,8159,False,K2100058159,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2021-05-8160,http://verkoop.domeinenrz.nl/verkoop_bij_insch...,Kavel K2100058160,0.0,False,"[Type 36, Kenteken WJ-89-JD, Datum eerste toel...",15,[http://verkoop.domeinenrz.nl/ufc/static/16194...,8160,False,K2100058160,...,NaN,NaN,249 kg,0 cm,NaN,537 cm,1.0,0.0,1.0,False
2021-05-8161,http://verkoop.domeinenrz.nl/verkoop_bij_insch...,Kavel K2100058161,2450.0,False,"[Type pk, Kenteken WD-VN-48, Datum eerste toel...",6,[http://verkoop.domeinenrz.nl/ufc/static/16194...,8161,False,K2100058161,...,NaN,NaN,1920 kg,0 cm,NaN,565 cm,1.0,0.0,1.0,False
2021-05-8162,http://verkoop.domeinenrz.nl/verkoop_bij_insch...,Kavel K2100058162,4565.0,False,"[Type movano, Kenteken V-546-DR, Afgelezen km-...",10,[http://verkoop.domeinenrz.nl/ufc/static/16194...,8162,False,K2100058162,...,0,Niet geregistreerd,1286 kg,0 cm,NaN,NaN,1.0,1.0,2.0,False


### Consolidate format
Older files are slightly different such as naming conventions etc. Here all results are consolidated into one format.

In [5]:
# Change type of early auctions for images
if VERBOSE > 0:
    # print first lot
    print('Fields with images look like this:\n')

    df_ = pd.DataFrame(index=data.keys(), columns=['column', 'type', 'first row'])
    # df_
    for k in data:
        im_cols = data[k].columns[
            data[k].columns.str.lower().isin(['image', 'images'])
        ]
        example = data[k][im_cols[0]].values[0]
        df_.loc[k, 'type'] = str(type(example))
        df_.loc[k, 'column'] = im_cols[0]
        if isinstance(example, str):
            df_.loc[k, 'first row'] = example
        else:
            df_.loc[k, 'first row'] = len(example)

    display(df_)


if OPBOD:
    # reformatting did not happen in IRS auctions
    pass
else:
    
    # convert string representation of list to real list
    # "['image1', 'image2']"
    for k in ['2017-03',
              '2017-04',
              '2017-05',
              '2017-06',
              '2017-07',
              '2017-08',
              '2017-09',
              '2017-10']:
        data[k]['Images'] = data[k]['Image'].apply(eval)
        data[k] = data[k].drop(columns=['Image'])

    # convert string representation of list without quotes to real list
    # "[image1, image2]"
    for k in ['2017-11',
              '2017-12',
              '2018-01',
              '2018-02',
              '2018-03',
              '2018-04']:
        data[k]['Images'] = data[k]['Images'].apply(lambda s:eval(re.sub(r"^\[(.*)\]$",r"['\1']",re.sub(' *, *',"', '",s))))
    

# Other inconsistencies
if OPBOD:
    pass
else:
    # Other column name that changed in November 2017
    for k in ['2017-03',
              '2017-04',
              '2017-05',
              '2017-06',
              '2017-07',
              '2017-08',
              '2017-09',
              '2017-10']:
        data[k] = data[k].rename(columns={
            'draw':'Draw',
            'nr':'LotNr',
            'raw':'Raw_text',
            'misc':'SupInfo',
            'footnote':'Note',
            'jfq':'jfc',
        })

Fields with images look like this:



,column,type,first row
2017-03,Image,<class 'str'>,['http://www.domeinenrz.nl/ufc/static/14883593...
2017-04,Image,<class 'str'>,['http://www.domeinenrz.nl/ufc/static/14906980...
2017-05,Image,<class 'str'>,['http://www.domeinenrz.nl/ufc/static/14931056...
2017-06,Image,<class 'str'>,['http://www.domeinenrz.nl/ufc/static/14962307...
2017-07,Image,<class 'str'>,['http://www.domeinenrz.nl/ufc/static/14986361...
2017-08,Image,<class 'str'>,['http://www.domeinenrz.nl/ufc/static/15016788...
2017-09,Image,<class 'str'>,['http://www.domeinenrz.nl/ufc/static/15040918...
2017-10,Image,<class 'str'>,['http://www.domeinenrz.nl/ufc/static/15065087...
2017-11,Images,<class 'str'>,[http://www.domeinenrz.nl/ufc/static/150952799...
2017-12,Images,<class 'str'>,[http://www.domeinenrz.nl/ufc/static/151195038...


# Merge all auctions into one data frame

In [6]:
all_data = pd.concat(data)
df = all_data.droplevel(0)

In [7]:
if VERBOSE > 0:
    # Show columns added
    col_exist = pd.DataFrame.from_dict({k:all_data.columns.isin(d.columns) for k,d in data.items()}, orient='index', columns=all_data.columns)
    col_filled = all_data.isna().all(level=0, axis=0) == False

    idx = col_exist.apply(lambda x:np.where(x)[0])

    first_occurence = col_exist.index[idx.apply(lambda x: x[0])]
    last_occurence = col_exist.index[idx.apply(lambda x: x[-1])]
    nr_occurence = idx.apply(len)
    pct_occurence = idx.apply(lambda x: 100*(len(x)/(col_exist.shape[0]-x[0])))

    idx = col_filled.apply(lambda x:np.where(x)[0])
    last_filled = col_filled.index[idx.apply(lambda x: x[-1] if len(x)>0 else col_exist.shape[0]-1)]
    nr_filled = idx.apply(len)
    pct_filled = idx.apply(lambda x: 100*(len(x)/(col_exist.shape[0]-x[0])) if len(x)>0 else 0)


    df_ = pd.DataFrame(zip(first_occurence, last_occurence, nr_occurence, last_filled, nr_filled, pct_occurence, pct_filled), 
                       index=col_exist.columns, 
                       columns=['first', 'last', 'n', 'last_filled', 'n_not_empty', 'pct_since_first', 'pct_filled'])



    for _by_rdw, df__ in df_.groupby(df_.index.str.startswith('rdw')):
        for g,df___ in df__.groupby('first'):
            if g == col_exist.index[0]:
                continue
            print(f'Added in {g}')
            if _by_rdw:
                print('by api')
            with pd.option_context("display.max_rows", 999):
                display(df___.sort_values(by='last_filled', ascending=False))

Added in 2017-11


,first,last,n,last_filled,n_not_empty,pct_since_first,pct_filled
N_images,2017-11,2021-05,41,2021-05,41,100.000000,100.000000
Source,2017-11,2021-05,41,2021-05,41,100.000000,100.000000
early_reg,2017-11,2021-05,41,2021-05,41,100.000000,100.000000
locked,2017-11,2021-05,28,2021-05,28,68.292683,68.292683
wo_frame,2017-11,2021-05,40,2021-05,40,97.560976,97.560976
used_parts,2017-11,2017-11,1,2017-11,1,2.439024,2.439024


Added in 2018-01


,first,last,n,last_filled,n_not_empty,pct_since_first,pct_filled
disclaim_cr6,2018-01,2021-05,33,2021-05,33,84.615385,84.615385
import22_btw21,2018-01,2018-07,2,2018-07,2,5.128205,5.128205
import27_btw21,2018-01,2018-07,2,2018-07,2,5.128205,5.128205


Added in 2018-04


...

,first,last,n,last_filled,n_not_empty,pct_since_first,pct_filled
rdw_brandstof_emis_co2_gewogen_gecombineerd_wltp_1,2021-03,2021-03,1,2021-05,0,33.333333,0.000000
rdw_brandstof_actie_radius_extern_opladen_wltp_2,2021-03,2021-03,1,2021-05,0,33.333333,0.000000
rdw_brandstof_actie_radius_extern_opladen_stad_wltp_2,2021-03,2021-03,1,2021-05,0,33.333333,0.000000
rdw_uitvoeringverbruik_per_uitgave_verbruikcategorie_uitvoering_21,2021-03,2021-05,2,2021-05,2,66.666667,66.666667
rdw_brandstof_elektrisch_verbruik_extern_opladen_wltp_2,2021-03,2021-03,1,2021-05,0,33.333333,0.000000
rdw_handelsbenaming_uitvoering_handelsbenaming_fabrikant_125,2021-03,2021-03,1,2021-03,1,33.333333,33.333333
rdw_handelsbenaming_uitvoering_handelsbenaming_fabrikant_126,2021-03,2021-03,1,2021-03,1,33.333333,33.333333
rdw_handelsbenaming_uitvoering_handelsbenaming_fabrikant_127,2021-03,2021-03,1,2021-03,1,33.333333,33.333333
rdw_handelsbenaming_uitvoering_registratiedatum_handelsbenaming_115,2021-03,2021-03,1,2021-03,1,33.333333,33.333333
rdw_handelsbenaming_uitvoering_registratiedatum_handelsbenaming_116,2021-03,2021-03,1,2021-03,1,33.333333,33.333333


Added in 2021-04
by api


,first,last,n,last_filled,n_not_empty,pct_since_first,pct_filled
rdw_assen_spoorbreedte_4,2021-04,2021-04,1,2021-05,0,50.0,0.0
rdw_assen_technisch_toegestane_maximum_aslast_4,2021-04,2021-04,1,2021-05,0,50.0,0.0
rdw_assen_weggedrag_code_4,2021-04,2021-04,1,2021-05,0,50.0,0.0
rdw_ovi_tenaamstellen_mogelijk,2021-04,2021-05,2,2021-05,2,100.0,100.0
rdw_assen_aantal_assen_4,2021-04,2021-04,1,2021-04,1,50.0,50.0
rdw_assen_wettelijk_toegestane_maximum_aslast_4,2021-04,2021-04,1,2021-04,1,50.0,50.0
rdw_assen_aangedreven_as_4,2021-04,2021-04,1,2021-04,1,50.0,50.0
rdw_assen_hefas_4,2021-04,2021-04,1,2021-04,1,50.0,50.0
rdw_assen_plaatscode_as_4,2021-04,2021-04,1,2021-04,1,50.0,50.0


Added in 2021-05
by api


,first,last,n,last_filled,n_not_empty,pct_since_first,pct_filled
rdw_tellerstandoordeel,2021-05,2021-05,1,2021-05,1,100.0,100.0
rdw_code_toelichting_tellerstandoordeel,2021-05,2021-05,1,2021-05,1,100.0,100.0
rdw_tenaamstellen_mogelijk,2021-05,2021-05,1,2021-05,1,100.0,100.0
rdw_jaar_laatste_registratie_tellerstand,2021-05,2021-05,1,2021-05,1,100.0,100.0


- - - -
# Continue with cars only

In [8]:
# select data
car=df[df.LotType == 'Personenauto'].copy()

if VERBOSE > 0:
    # print lots that are cars per auction
    df_ = pd.DataFrame(columns=['year', 'month', 'lot'])
    df_.year = pd.Series(car.index).apply(lambda x: int(x.split('-')[0]))
    df_.month = pd.Series(car.index).apply(lambda x: int(x.split('-')[1]))
    df_.lot = pd.Series(car.index).apply(lambda x: int(x.split('-')[2]))
    display(df_.groupby(['year', 'month']).lot.agg(['min', 'max', 'count']).sort_index())
    
car.tail()

min   max  count
year month                   
2017 3      2000  8318    149
     4      2000  7309    133
     5      2200  8025     75
     6      2000  7311    140
     7      2200  8178    106
     8      2001  7304    143
     9      2200  8187    133
     10     2000  7171     95
     11     2200  8305    158
     12     2000  7308    136
2018 1      2200  8170    134
     2      2000  7184    138
     3      2200  8338    186
     4      2000  7336    126
     5      2200  8226    186
     6      2000  7315    166
     7      2200  8311    132
     8      2000  7328    152
     9      2200  8183     89
     10     2000  7319     87
     11     2200  8326    127
     12     2000  7361    133
2019 1      2201  8329    135
     2      2001  7264    176
     3      2200  8318    145
     4      2000  7314    170
     5      2200  8352    158
     6      2201  9158     98
     7      2200  7254    144
     8      2200  8172     96
     9      2200  9269    156
     10     2400  7200    109
     11     2200  8213    125
     12     2200  9061     75
2020 1      2200  7205    126
     2      2200  8238    129
     3      2200  9152     65
     6      2200  7269    160
     7      2200  9271    145
     8      2200  7286    187
     9      2200  8236    111
     10     2200  7266    129
     11     2200  8260    121
     12     2201  7306    168
2021 1      2200  8123    104
     2      2200  7156    129
     3      2200  8189    148
     4      2200  7220    173
     5      2200  8158    126

,LotNr,Price,Title,LotCat,LotType,ItemBrand,ItemType,Mfdate,Mfyear,APKdate,...,rdw_assen_aangedreven_as_4,rdw_assen_hefas_4,rdw_assen_plaatscode_as_4,rdw_assen_weggedrag_code_4,rdw_ovi_tenaamstellen_mogelijk,disclaim6,rdw_tellerstandoordeel,rdw_code_toelichting_tellerstandoordeel,rdw_tenaamstellen_mogelijk,rdw_jaar_laatste_registratie_tellerstand
2021-05-8152,K2100058152,13456.0,Kavel K2100058152,Voertuigen en onderdelen,Personenauto,OPEL,insignia grand sport,20.07.2017,,27.07.2021,...,NaN,NaN,NaN,NaN,Ja,False,Geen oordeel,05,Ja,2020.0
2021-05-8154,K2100058154,3255.0,Kavel K2100058154,Voertuigen en onderdelen,Personenauto,DACIA,duster,30.08.2013,,,...,NaN,NaN,NaN,NaN,NaN,False,NaN,NaN,NaN,NaN
2021-05-8155,K2100058155,6064.0,Kavel K2100058155,Voertuigen en onderdelen,Personenauto,VOLKSWAGEN,passat,10.01.2013,,,...,NaN,NaN,NaN,NaN,Ja,False,Geen oordeel,05,Ja,2019.0
2021-05-8157,K2100058157,0.0,Kavel K2100058157,Voertuigen en onderdelen,Personenauto,HYUNDAI,i30,02.10.2014,,,...,NaN,NaN,NaN,NaN,NaN,False,NaN,NaN,NaN,NaN
2021-05-8158,K2100058158,2222.0,Kavel K2100058158,Voertuigen en onderdelen,Personenauto,LANDROVER,range rover sport,30.01.2006,,,...,NaN,NaN,NaN,NaN,NaN,False,NaN,NaN,NaN,NaN


# Save file

In [9]:
if OPBOD:
    file_name = r'../../../python-nb/data/cars-from-all-auctions-opbod.pkl'
else:
    file_name = r'../data/cars-from-all-auctions.pkl'

if (SKIPSAVE==False): #and (not(os.path.isfile(file_name))):
    print(file_name)
    car.to_pickle(file_name)
else:
    print(f'Skip. {file_name} exists or saving is disabled in settings.')
    


../data/cars-from-all-auctions.pkl


# Write example results to file
based on: https://stackoverflow.com/a/33869154

In [10]:
def pandas_df_to_markdown_table(df):
    fmt = ['-----' for i in range(len(df.columns))]
    df_fmt = pd.DataFrame([fmt], columns=df.columns)
    df_formatted = pd.concat([df_fmt, df])
    return df_formatted.to_csv(sep="|", index=False)


In [11]:
file_name = '../assets/example-table-of-cars.md'

example = car.tail(10).copy()

# trim some long fields
example.rdwinfo = '.. rdw info ..'
example.Raw_text = '.. raw text ..'
example.SupInfo = '.. suplm. info. ..'

# convert to md
try:
    table_text = example.to_markdown()
except ImportError:
    print('Fallback')
    table_text = pandas_df_to_markdown_table(example)

# save
if (SKIPSAVE==False) and (not(os.path.isfile(file_name))):
    with open(file_name,'w') as file:
        file.write(table_text)

    print('A markdown table is available as\n\t{}'.format(file_name))
else:
    print(f'Skip. {file_name} exists or saving is disabled in settings.')
    display(example)
    


Skip. ../assets/example-table-of-cars.md exists or saving is disabled in settings.


,LotNr,Price,Title,LotCat,LotType,ItemBrand,ItemType,Mfdate,Mfyear,APKdate,...,rdw_assen_aangedreven_as_4,rdw_assen_hefas_4,rdw_assen_plaatscode_as_4,rdw_assen_weggedrag_code_4,rdw_ovi_tenaamstellen_mogelijk,disclaim6,rdw_tellerstandoordeel,rdw_code_toelichting_tellerstandoordeel,rdw_tenaamstellen_mogelijk,rdw_jaar_laatste_registratie_tellerstand
2021-05-8147,K2100058147,8639.0,Kavel K2100058147,Voertuigen en onderdelen,Personenauto,MERCEDES-BENZ,citan,25.02.2016,,,...,NaN,NaN,NaN,NaN,Ja,False,Logisch,00,Ja,2021.0
2021-05-8148,K2100058148,0.0,Kavel K2100058148,Voertuigen en onderdelen,Personenauto,RENAULT,megane,04.10.2006,,22.12.2021,...,NaN,NaN,NaN,NaN,Ja,False,Logisch,00,Ja,2021.0
2021-05-8149,K2100058149,3189.0,Kavel K2100058149,Voertuigen en onderdelen,Personenauto,MERCEDES-BENZ,clk 200 kompressor; coupe,16.02.2005,,,...,NaN,NaN,NaN,NaN,Ja,False,Logisch,00,Ja,2021.0
2021-05-8150,K2100058150,4189.0,Kavel K2100058150,Voertuigen en onderdelen,Personenauto,PEUGEOT,208 5p style 1.,,onbekend,,...,NaN,NaN,NaN,NaN,NaN,False,NaN,NaN,NaN,NaN
2021-05-8151,K2100058151,677.0,Kavel K2100058151,Voertuigen en onderdelen,Personenauto,FORD,mondeo zetec b-,30.05.2013,,,...,NaN,NaN,NaN,NaN,NaN,False,NaN,NaN,NaN,NaN
2021-05-8152,K2100058152,13456.0,Kavel K2100058152,Voertuigen en onderdelen,Personenauto,OPEL,insignia grand sport,20.07.2017,,27.07.2021,...,NaN,NaN,NaN,NaN,Ja,False,Geen oordeel,05,Ja,2020.0
2021-05-8154,K2100058154,3255.0,Kavel K2100058154,Voertuigen en onderdelen,Personenauto,DACIA,duster,30.08.2013,,,...,NaN,NaN,NaN,NaN,NaN,False,NaN,NaN,NaN,NaN
2021-05-8155,K2100058155,6064.0,Kavel K2100058155,Voertuigen en onderdelen,Personenauto,VOLKSWAGEN,passat,10.01.2013,,,...,NaN,NaN,NaN,NaN,Ja,False,Geen oordeel,05,Ja,2019.0
2021-05-8157,K2100058157,0.0,Kavel K2100058157,Voertuigen en onderdelen,Personenauto,HYUNDAI,i30,02.10.2014,,,...,NaN,NaN,NaN,NaN,NaN,False,NaN,NaN,NaN,NaN
2021-05-8158,K2100058158,2222.0,Kavel K2100058158,Voertuigen en onderdelen,Personenauto,LANDROVER,range rover sport,30.01.2006,,,...,NaN,NaN,NaN,NaN,NaN,False,NaN,NaN,NaN,NaN
